    Esse projeto tem a função de:

    1-Criar um sistema TKINTER que:
   

        conforme link https://www.researchgate.net/figure/Figura-2-Simbolos-para-fluxograma-Fonte-Os-autores_fig1_325917131

        
    2-Deve ter as seguintes opções de input de atividades: 

    Atividade = Retângulo horizontal

    Inspeção = Círculo

    Adiamento = D semicírculo ou símbolo de espera

    Armazenagem = Triângulo invertido

    Decisão = Losango

    Início = Oval ou elipse

    Fim = Oval ou elipse

    Fluxo = Linha ou seta

    Documento = Retângulo com uma linha ondulada na parte inferior

    Nome do banco de dados = Cilindro

    Movimentação = Linha ou seta contínua

    3-As opções de ipnut terá a função de armazenar qualquer nome aleatório que será escrito manualmente e também uma text box adicional para escolher uma das opções de Input;

    4-O sistema TKINTER deverá ter um botão de confirmação que vai gerar uma imagem no próprio sistema TKINTER do workflow ou fluxograma alimentados com as informações incluídas em opções de Input;

    4B-A visualização do Fluxograma ou workflow deve seguir uma vizualização de fácil entendimento humano, ou seja o início começa na esquerda e o fim na direita, podendo ser de cima para baixo ou 
    a melhor forma de visualização que você puder colocar mas atendendo a lógica que o começo é na esquerda e o fim na direita.

    5-O sistema TKINTER deverá ter a função de criar uma descrição do problema a ser resolvido com base nas informações incluídas no input de atividades, criando assim um texto automático,
    para que eu possa ler qual é o problema a ser resolvido;

    6-Deve ter um botão para limpar as informações do sistema;

    7-Deve ter um botão para escolher onde será incluido o elemento será disposto no canvas, se é na esquerda, direita abaixo ou acima do último canvas inserido, com exceção do elemento de inicio que sempre vai começar na esquerda no topo do canvas;

    8-A conexão vai ser feita na forma de uma linha ou seta (Fluxo) sem mover os elementos.

In [1]:
import streamlit as st
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import io
import base64
from PIL import Image, ImageDraw
import pandas as pd
import numpy as np

# Configuração inicial da página
st.set_page_config(page_title="Sistema de Fluxograma", layout="wide")

# Inicialização de variáveis de estado
if 'elementos' not in st.session_state:
    st.session_state.elementos = []
    
if 'conexoes' not in st.session_state:
    st.session_state.conexoes = []
    
if 'ultimo_elemento' not in st.session_state:
    st.session_state.ultimo_elemento = None
    
if 'last_pos' not in st.session_state:
    st.session_state.last_pos = {"x": 0, "y": 0}

# Funções auxiliares para geração de formas
def criar_forma_elemento(tipo, tamanho=50):
    img = Image.new('RGBA', (tamanho, tamanho), (255, 255, 255, 0))
    draw = ImageDraw.Draw(img)
    
    if tipo == "Atividade":
        # Retângulo horizontal
        draw.rectangle([(0, 10), (tamanho, tamanho-10)], fill=(173, 216, 230, 255), outline=(0, 0, 0, 255))
    elif tipo == "Inspeção":
        # Círculo
        draw.ellipse([(5, 5), (tamanho-5, tamanho-5)], fill=(144, 238, 144, 255), outline=(0, 0, 0, 255))
    elif tipo == "Adiamento":
        # D semicírculo
        draw.pieslice([(5, 5), (tamanho-5, tamanho-5)], 270, 90, fill=(255, 165, 0, 255), outline=(0, 0, 0, 255))
    elif tipo == "Armazenagem":
        # Triângulo invertido
        draw.polygon([(5, 5), (tamanho-5, 5), (tamanho/2, tamanho-5)], fill=(255, 255, 0, 255), outline=(0, 0, 0, 255))
    elif tipo == "Decisão":
        # Losango
        draw.polygon([(tamanho/2, 5), (tamanho-5, tamanho/2), (tamanho/2, tamanho-5), (5, tamanho/2)], 
                      fill=(255, 182, 193, 255), outline=(0, 0, 0, 255))
    elif tipo == "Início":
        # Oval
        draw.ellipse([(5, 10), (tamanho-5, tamanho-10)], fill=(144, 238, 144, 255), outline=(0, 0, 0, 255))
    elif tipo == "Fim":
        # Oval
        draw.ellipse([(5, 10), (tamanho-5, tamanho-10)], fill=(250, 128, 114, 255), outline=(0, 0, 0, 255))
    elif tipo == "Documento":
        # Retângulo com linha ondulada
        draw.rectangle([(5, 5), (tamanho-5, tamanho-10)], fill=(255, 255, 224, 255), outline=(0, 0, 0, 255))
        # Linha ondulada
        pontos = []
        for i in range(5, tamanho-5, 4):
            pontos.extend([i, tamanho-5 + (3 if i % 8 == 5 else 0)])
        draw.line(pontos, fill=(0, 0, 0, 255), width=1)
    elif tipo == "Banco de Dados":
        # Cilindro
        draw.ellipse([(5, 5), (tamanho-5, 15)], fill=(224, 255, 255, 255), outline=(0, 0, 0, 255))
        draw.rectangle([(5, 10), (tamanho-5, tamanho-10)], fill=(224, 255, 255, 255), outline=(0, 0, 0, 255))
        draw.ellipse([(5, tamanho-20), (tamanho-5, tamanho-10)], fill=(224, 255, 255, 255), outline=(0, 0, 0, 255))
    
    return img

# Função para adicionar elemento
def adicionar_elemento(nome, tipo, posicao):
    if not nome:
        st.error("Por favor, insira um nome para o elemento.")
        return False
    
    # Definir posição inicial para o primeiro elemento (Início)
    if not st.session_state.elementos:
        if tipo != "Início":
            st.error("O primeiro elemento deve ser do tipo 'Início'.")
            return False
        pos_x, pos_y = 0, 0
    else:
        # Calcular posição com base no último elemento e na posição escolhida
        spacing_x = 1
        spacing_y = 1
        
        if st.session_state.ultimo_elemento:
            ultimo = st.session_state.ultimo_elemento
            pos_x = ultimo["pos_x"]
            pos_y = ultimo["pos_y"]
            
            if posicao == "Direita":
                pos_x += spacing_x
            elif posicao == "Abaixo":
                pos_y += spacing_y
            elif posicao == "Esquerda":
                pos_x = max(0, pos_x - spacing_x)
            elif posicao == "Acima":
                pos_y = max(0, pos_y - spacing_y)
    
    # Verificar se já existe um elemento com esse nome
    for elem in st.session_state.elementos:
        if elem["nome"] == nome:
            st.error(f"Já existe um elemento com o nome '{nome}'.")
            return False
    
    # Criar novo elemento
    elemento = {
        "id": len(st.session_state.elementos),
        "nome": nome,
        "tipo": tipo,
        "pos_x": pos_x,
        "pos_y": pos_y,
        "conexoes": []
    }
    
    st.session_state.elementos.append(elemento)
    st.session_state.ultimo_elemento = elemento
    st.session_state.last_pos = {"x": pos_x, "y": pos_y}
    
    return True

# Função para conectar elementos
def conectar_elementos(origem_id, destino_id):
    if origem_id == destino_id:
        st.error("Origem e destino não podem ser o mesmo elemento.")
        return False
    
    # Verificar se a conexão já existe
    for origem, destino in st.session_state.conexoes:
        if origem == origem_id and destino == destino_id:
            st.error("Esta conexão já existe.")
            return False
    
    # Adicionar conexão
    st.session_state.conexoes.append((origem_id, destino_id))
    
    # Atualizar as conexões no elemento de origem
    for elem in st.session_state.elementos:
        if elem["id"] == origem_id:
            elem["conexoes"].append(destino_id)
            break
    
    return True

# Função para gerar descrição do fluxograma
def gerar_descricao():
    if not st.session_state.elementos:
        st.error("Não há elementos no fluxograma para gerar uma descrição.")
        return ""
    
    # Encontrar elemento inicial
    inicio = None
    for elemento in st.session_state.elementos:
        if elemento["tipo"] == "Início":
            inicio = elemento
            break
    
    if not inicio:
        st.error("Não foi encontrado um elemento do tipo 'Início'.")
        return ""
    
    # Gerar descrição do fluxograma
    descricao = "Descrição do Fluxograma:\n\n"
    descricao += f"Este fluxograma inicia com '{inicio['nome']}' e "
    
    # Contar tipos de elementos
    tipos_count = {}
    for elemento in st.session_state.elementos:
        if elemento["tipo"] in tipos_count:
            tipos_count[elemento["tipo"]] += 1
        else:
            tipos_count[elemento["tipo"]] = 1
    
    descricao += "contém:\n"
    for tipo, count in tipos_count.items():
        descricao += f"- {count} elemento(s) do tipo '{tipo}'\n"
    
    descricao += f"\nTotal de {len(st.session_state.elementos)} elementos e {len(st.session_state.conexoes)} conexões.\n\n"
    
    # Tentar traçar um caminho a partir do início
    visitados = set()
    caminho = []
    
    def trace_path(elemento_id):
        if elemento_id in visitados:
            return
        
        visitados.add(elemento_id)
        
        # Encontrar o elemento pelo ID
        elemento = next((e for e in st.session_state.elementos if e["id"] == elemento_id), None)
        if elemento:
            caminho.append(elemento)
            
            for proximo_id in elemento["conexoes"]:
                trace_path(proximo_id)
    
    trace_path(inicio["id"])
    
    # Adicionar descrição do caminho
    if len(caminho) > 1:
        descricao += "Sequência de execução:\n"
        for i, elemento in enumerate(caminho):
            descricao += f"{i+1}. {elemento['nome']} ({elemento['tipo']})"
            if i < len(caminho) - 1:
                descricao += " → "
            if (i + 1) % 3 == 0:  # Quebra de linha a cada 3 elementos
                descricao += "\n"
    
    # Elementos não visitados (possivelmente desconectados)
    ids_visitados = {e["id"] for e in caminho}
    nao_visitados = [e for e in st.session_state.elementos if e["id"] not in ids_visitados]
    
    if nao_visitados:
        descricao += "\n\nElementos não conectados ao fluxo principal:\n"
        for elemento in nao_visitados:
            descricao += f"- {elemento['nome']} ({elemento['tipo']})\n"
    
    # Decisões e suas ramificações
    decisoes = [e for e in st.session_state.elementos if e["tipo"] == "Decisão"]
    if decisoes:
        descricao += "\nPontos de decisão:\n"
        for decisao in decisoes:
            descricao += f"- '{decisao['nome']}' pode levar a: "
            destinos = []
            for elem in st.session_state.elementos:
                if elem["id"] in decisao["conexoes"]:
                    destinos.append(f"'{elem['nome']}'")
            
            if destinos:
                descricao += ", ".join(destinos)
            else:
                descricao += "nenhum destino conectado"
            descricao += "\n"
    
    return descricao

# Função para desenhar o fluxograma
def desenhar_fluxograma():
    if not st.session_state.elementos:
        st.info("Adicione elementos para visualizar o fluxograma.")
        return
    
    # Criar um gráfico direcionado
    G = nx.DiGraph()
    
    # Definir nós e posições
    pos = {}
    node_colors = []
    node_shapes = []
    node_sizes = []
    
    for elemento in st.session_state.elementos:
        G.add_node(elemento["id"], label=elemento["nome"], tipo=elemento["tipo"])
        pos[elemento["id"]] = (elemento["pos_x"], -elemento["pos_y"])  # Invertendo Y para visualização de cima para baixo
        
        # Definir cores dos nós por tipo
        if elemento["tipo"] == "Atividade":
            node_colors.append('lightblue')
            node_shapes.append('s')  # square
            node_sizes.append(1000)
        elif elemento["tipo"] == "Inspeção":
            node_colors.append('lightgreen')
            node_shapes.append('o')  # circle
            node_sizes.append(800)
        elif elemento["tipo"] == "Adiamento":
            node_colors.append('orange')
            node_shapes.append('h')  # hexagon
            node_sizes.append(800)
        elif elemento["tipo"] == "Armazenagem":
            node_colors.append('yellow')
            node_shapes.append('v')  # triangle down
            node_sizes.append(800)
        elif elemento["tipo"] == "Decisão":
            node_colors.append('lightpink')
            node_shapes.append('d')  # diamond
            node_sizes.append(1000)
        elif elemento["tipo"] == "Início":
            node_colors.append('lightgreen')
            node_shapes.append('o')  # circle
            node_sizes.append(800)
        elif elemento["tipo"] == "Fim":
            node_colors.append('salmon')
            node_shapes.append('o')  # circle
            node_sizes.append(800)
        elif elemento["tipo"] == "Documento":
            node_colors.append('lightyellow')
            node_shapes.append('s')  # square
            node_sizes.append(1000)
        elif elemento["tipo"] == "Banco de Dados":
            node_colors.append('lightcyan')
            node_shapes.append('8')  # octagon
            node_sizes.append(800)
        else:
            node_colors.append('white')
            node_shapes.append('o')  # circle
            node_sizes.append(800)
    
    # Adicionar arestas
    for origem_id, destino_id in st.session_state.conexoes:
        G.add_edge(origem_id, destino_id)
    
    # Criar figura
    plt.figure(figsize=(12, 8))
    
    # Desenhar nós com diferentes formas
    for shape in set(node_shapes):
        node_list = [node for node, node_shape in zip(G.nodes(), node_shapes) if node_shape == shape]
        node_color_list = [node_colors[i] for i, node in enumerate(G.nodes()) if node in node_list]
        node_size_list = [node_sizes[i] for i, node in enumerate(G.nodes()) if node in node_list]
        
        if shape == 'd':  # Para losangos (decisões)
            nx.draw_networkx_nodes(G, pos, nodelist=node_list, node_color=node_color_list, 
                                 node_shape=shape, node_size=node_size_list, edgecolors='black')
        else:
            nx.draw_networkx_nodes(G, pos, nodelist=node_list, node_color=node_color_list, 
                                 node_shape=shape, node_size=node_size_list, edgecolors='black')
    
    # Desenhar arestas e rótulos
    nx.draw_networkx_edges(G, pos, arrows=True, arrowsize=20, width=1.5)
    
    # Obter labels
    labels = {node: G.nodes[node]['label'] for node in G.nodes()}
    nx.draw_networkx_labels(G, pos, labels=labels, font_size=9, font_family='sans-serif')
    
    # Criar legenda
    tipo_cores = {
        "Início": 'lightgreen',
        "Atividade": 'lightblue',
        "Inspeção": 'lightgreen',
        "Adiamento": 'orange',
        "Armazenagem": 'yellow',
        "Decisão": 'lightpink',
        "Fim": 'salmon',
        "Documento": 'lightyellow',
        "Banco de Dados": 'lightcyan'
    }
    
    patches = [mpatches.Patch(color=color, label=tipo) for tipo, color in tipo_cores.items() 
              if tipo in [elem["tipo"] for elem in st.session_state.elementos]]
    
    plt.legend(handles=patches, loc='upper right')
    plt.axis('off')
    plt.tight_layout()
    
    # Salvar figura e mostrar
    buf = io.BytesIO()
    plt.savefig(buf, format='png', dpi=300, bbox_inches='tight')
    buf.seek(0)
    
    return Image.open(buf)

# Função para limpar tudo
def limpar_tudo():
    st.session_state.elementos = []
    st.session_state.conexoes = []
    st.session_state.ultimo_elemento = None
    st.session_state.last_pos = {"x": 0, "y": 0}
    st.success("Fluxograma limpo com sucesso!")

# Interface principal
st.title("Sistema de Fluxograma")

# Layout em colunas
col1, col2 = st.columns([1, 2])

with col1:
    st.header("Controles")
    
    # Formulário para adicionar elemento
    with st.form(key="adicionar_elemento_form"):
        nome_elemento = st.text_input("Nome do Elemento")
        
        tipos_elementos = [
            "Início", "Atividade", "Inspeção", "Adiamento", "Armazenagem", 
            "Decisão", "Documento", "Banco de Dados", "Fim"
        ]
        tipo_elemento = st.selectbox("Tipo do Elemento", tipos_elementos)
        
        posicoes = ["Direita", "Abaixo", "Esquerda", "Acima"]
        posicao_elemento = st.selectbox("Posição do Elemento", posicoes)
        
        submit_button = st.form_submit_button(label="Adicionar Elemento")
        
        if submit_button:
            if adicionar_elemento(nome_elemento, tipo_elemento, posicao_elemento):
                st.success(f"Elemento '{nome_elemento}' adicionado com sucesso!")
    
    # Formulário para conectar elementos
    if len(st.session_state.elementos) >= 2:
        with st.form(key="conectar_elementos_form"):
            st.subheader("Conectar Elementos")
            
            origem_options = [f"{elem['nome']} ({elem['tipo']})" for elem in st.session_state.elementos]
            origem_idx = st.selectbox("De:", range(len(origem_options)), format_func=lambda x: origem_options[x])
            
            destino_options = [f"{elem['nome']} ({elem['tipo']})" for elem in st.session_state.elementos]
            destino_idx = st.selectbox("Para:", range(len(destino_options)), format_func=lambda x: destino_options[x])
            
            submit_conexao = st.form_submit_button(label="Conectar")
            
            if submit_conexao:
                origem_id = st.session_state.elementos[origem_idx]["id"]
                destino_id = st.session_state.elementos[destino_idx]["id"]
                
                if conectar_elementos(origem_id, destino_id):
                    st.success("Elementos conectados com sucesso!")
    
    # Botões de ação
    col_btn1, col_btn2 = st.columns(2)
    
    with col_btn1:
        if st.button("Gerar Descrição"):
            st.session_state.descricao = gerar_descricao()
    
    with col_btn2:
        if st.button("Limpar Tudo"):
            limpar_tudo()
    
    # Área para a descrição gerada
    if 'descricao' in st.session_state and st.session_state.descricao:
        st.subheader("Descrição do Fluxograma")
        st.text_area("", st.session_state.descricao, height=300)

with col2:
    st.header("Visualização do Fluxograma")
    
    # Mostrar lista de elementos adicionados
    if st.session_state.elementos:
        with st.expander("Elementos Adicionados"):
            dados = []
            for elem in st.session_state.elementos:
                dados.append({
                    "ID": elem["id"],
                    "Nome": elem["nome"],
                    "Tipo": elem["tipo"],
                    "Conexões": len(elem["conexoes"])
                })
            st.dataframe(pd.DataFrame(dados))
        
        # Mostrar conexões
        if st.session_state.conexoes:
            with st.expander("Conexões"):
                conexoes_data = []
                for origem_id, destino_id in st.session_state.conexoes:
                    origem = next((e for e in st.session_state.elementos if e["id"] == origem_id), None)
                    destino = next((e for e in st.session_state.elementos if e["id"] == destino_id), None)
                    if origem and destino:
                        conexoes_data.append({
                            "De": f"{origem['nome']} ({origem['tipo']})",
                            "Para": f"{destino['nome']} ({destino['tipo']})"
                        })
                st.dataframe(pd.DataFrame(conexoes_data))
    
    # Visualizar fluxograma
    if st.session_state.elementos:
        fluxograma_img = desenhar_fluxograma()
        st.image(fluxograma_img, use_column_width=True)
    else:
        st.info("Adicione elementos ao fluxograma para visualizá-lo.")
    
    # Mostrar legenda de formas
    with st.expander("Legenda de Formas"):
        leg_col1, leg_col2, leg_col3 = st.columns(3)
        
        tipos_elementos = {
            "Atividade": "Retângulo horizontal",
            "Inspeção": "Círculo", 
            "Adiamento": "Semicírculo (D)",
            "Armazenagem": "Triângulo invertido",
            "Decisão": "Losango",
            "Início/Fim": "Oval ou elipse",
            "Documento": "Retângulo com ondulação",
            "Banco de Dados": "Cilindro",
            "Fluxo": "Linha ou seta"
        }
        
        for i, (tipo, descricao) in enumerate(tipos_elementos.items()):
            col = [leg_col1, leg_col2, leg_col3][i % 3]
            with col:
                if tipo != "Fluxo" and tipo != "Início/Fim":
                    img = criar_forma_elemento(tipo)
                    st.image(img, width=40, caption=f"{tipo}: {descricao}")
                elif tipo == "Início/Fim":
                    img = criar_forma_elemento("Início")
                    st.image(img, width=40, caption=f"{tipo}: {descricao}")
                else:
                    st.markdown(f"**{tipo}:** {descricao}")

st.sidebar.title("Sobre o Sistema")
st.sidebar.info("""
Este sistema permite criar fluxogramas com vários tipos de elementos e suas conexões.

**Como usar:**
1. Adicione elementos escolhendo nome, tipo e posição
2. Conecte os elementos entre si
3. Visualize o fluxograma gerado
4. Gere uma descrição automática do fluxograma

Desenvolvido com Streamlit e NetworkX.
""")

# Mostrar informações extras na barra lateral
st.sidebar.subheader("Estatísticas")
st.sidebar.write(f"Total de elementos: {len(st.session_state.elementos)}")
st.sidebar.write(f"Total de conexões: {len(st.session_state.conexoes)}")

if st.session_state.elementos:
    tipos_count = {}
    for elemento in st.session_state.elementos:
        if elemento["tipo"] in tipos_count:
            tipos_count[elemento["tipo"]] += 1
        else:
            tipos_count[elemento["tipo"]] = 1
    
    st.sidebar.subheader("Elementos por tipo")
    for tipo, count in tipos_count.items():
        st.sidebar.write(f"- {tipo}: {count}")

2025-02-26 12:51:01.236 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-26 12:51:01.237 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-26 12:51:01.238 WARNING streamlit.runtime.state.session_state_proxy: Session state does not function when running a script without `streamlit run`
2025-02-26 12:51:01.238 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-26 12:51:01.239 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-26 12:51:01.239 WARNING streamlit.runtime.scriptrunner_utils.script_run_c